In [ ]:
!pip install --upgrade pip --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00


In [1]:
!pip install transformers torch --quiet

In [ ]:
import os
from torch import cuda, bfloat16
from time import time
import transformers
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cuda.is_available()

True

In [4]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-chat-hf", "snapshots", "1")

In [ ]:
os.path.exists(model_id)


True

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 4.148 sec.


In [9]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=2000,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:


test_model(tokenizer,
           query_pipeline,
           "As a Genearal in the Moldovian army provide a 500 word detailed description of a Russian invasion of Moldova.")

